Experimental area


# **Importing Dependencies**

In [0]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import np_utils
import spacy
from keras.preprocessing.text import Tokenizer
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense,LSTM,Embedding
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.sequence import pad_sequences
import pickle


#**Loading Data**

In [6]:
#Cloning from Data Repo
!git clone https://github.com/anantdevcs/Data.git
%cd Data

Cloning into 'Data'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 18 (delta 5), reused 13 (delta 3), pack-reused 0
Unpacking objects: 100% (18/18), done.
/content/Data


In [7]:
#If some chnages happen to Data repo
!git pull

Already up to date.


In [8]:

text = open('hamlet.txt').read().lower()
text

'\ufeff***the project gutenberg\'s etext of shakespeare\'s first folio***\n*********************the tragedie of hamlet*********************\n\n\n*******************************************************************\nthis ebook was one of project gutenberg\'s early files produced at a\ntime when proofing methods and tools were not well developed. there\nis an improved edition of this title which may be viewed as ebook\n(#100) at https://www.gutenberg.org/ebooks/100\n*******************************************************************\n\n\nthis is our 3rd edition of most of these plays.  see the index.\n\n\ncopyright laws are changing all over the world, be sure to check\nthe copyright laws for your country before posting these files!!\n\nplease take a look at the important information in this header.\nwe encourage you to keep this file on your own disk, keeping an\nelectronic path open for the next readers.  do not remove this.\n\n\n**welcome to the world of free plain vanilla electronic t

#**Exploring Data**

In [0]:
nlp = spacy.load('en',disable=['parser', 'tagger','ner'])
nlp.max_length = 1198623

In [0]:
def separate_punc(doc_text):
    return [token.text.lower() for token in nlp(doc_text) if token.text not in '\n\n \n\n\n!"-#$%&()--.*+,-/:;<=>?@[\\]^_`{|}~\t\n ']

In [0]:
tokens = separate_punc(text)

In [0]:
sequences = []
SEQ = 16
for i in range(len(tokens) - SEQ-1):
  sequences.append(tokens[i:(i + SEQ+1)])


In [13]:
sequences[25]

['at',
 'a',
 'time',
 'when',
 'proofing',
 'methods',
 'and',
 'tools',
 'were',
 'not',
 'well',
 'developed',
 'there',
 'is',
 'an',
 'improved',
 'edition']

#**Data Preprocessing**

In [0]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sequences)
num_sequences = tokenizer.texts_to_sequences(sequences)


In [15]:
num_sequences = np.array(num_sequences)
print(num_sequences.shape)
total_words = len(tokenizer.word_index)

(33772, 17)


In [0]:
X = num_sequences[:,0:SEQ]
y = num_sequences[:,SEQ]
y = to_categorical(y, num_classes=total_words)


#**Creating a LSTM Model**

In [19]:
model = Sequential()
model.add(Embedding(total_words , SEQ, input_length=SEQ))
model.add(LSTM(700, return_sequences=True))
model.add(LSTM(700))
model.add(Dense(500, activation='relu'))
model.add(Dense(total_words, activation='softmax'))
model.summary()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 16, 16)            85296     
_________________________________________________________________
lstm_3 (LSTM)                (None, 16, 700)           2007600   
_________________________________________________________________
lstm_4 (LSTM)                (None, 700)               3922800   
_________________________________________________________________
dense_3 (Dense)              (None, 500)               350500    
_________________________________________________________________
dense_4 (Dense)              (None, 5331)              2670831   
Total params: 9,037,027
Trainable params: 9,037,027
Non-trainable params: 0
_________________________________________________________________


#**Training the model**

In [0]:
history = model.fit(X, y, batch_size=512, epochs=200,verbose=1, callbacks=callbacks_list)

Epoch 1/200
33772/33772 [==============================] - 9s 261us/step - loss: 7.0228 - acc: 0.0281
Epoch 2/200
33772/33772 [==============================] - 7s 213us/step - loss: 6.6049 - acc: 0.0321
Epoch 3/200
33772/33772 [==============================] - 7s 211us/step - loss: 6.5149 - acc: 0.0321
Epoch 4/200
13824/33772 [===========>..................] - ETA: 4s - loss: 6.4091 - acc: 0.0362

In [0]:
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min', period=50)
callbacks_list = [checkpoint]

#**Analysing the training stats**

#**Trying to generate some random text**

In [0]:
seed = "But when in thee time's furrows I behold,  Then look I death my days should expiate.  For all that beauty that doth cover thee,  Is but the seemly raiment of my heart, "
output_text = []
NUM_WORD_GEN = 100
for i in range(NUM_WORD_GEN):
  encoded_text = tokenizer.texts_to_sequences([seed])[0]
  pad_encoded = pad_sequences([encoded_text], maxlen=SEQ, truncating='pre')
  pred_word_ind = model.predict_classes(pad_encoded, verbose=0)[0]
  pred_word = tokenizer.index_word[pred_word_ind] 
  seed += ' ' + pred_word
  output_text.append(pred_word)

print(output_text)


#**Saving the model and the Tokenizer**

In [0]:
model.save('model.h5')
token_save = open('token_dump', 'ab')       
# source, destination 
pickle.dump(tokenizer, token_save)                      
token_save.close() 
